# Neural Operator Baseline for Heat Equation
This notebook implements a Fourier Neural Operator (FNO) model using neuraloperator to solve the 2D heat equation.

In [20]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from datetime import datetime
import logging
from neuralop.models import TFNO2d
from neuralop import Trainer

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Check for GPU availability and configure device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    logger.info(f"Using GPU: {torch.cuda.get_device_name()}")
else:
    logger.warning("CUDA is not available, using CPU")

INFO:__main__:Using GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [21]:
# Define parameters
nx = 30  # Number of points in x direction
ny = 30  # Number of points in y direction
alpha = 0.05  # Diffusion coefficient
t1 = 0  # Initial temperature at x=0
t2 = 1  # Initial temperature at x=1
end_time = 1  # End time

In [22]:
# Generate training data
def generate_data(n_samples=1000):
    # Create grid
    x = np.linspace(0, 1, nx)
    y = np.linspace(0, 1, ny)
    t = np.linspace(0, end_time, 50)
    X, Y = np.meshgrid(x, y)
    
    # Initialize data arrays
    inputs = np.zeros((n_samples, 1, nx, ny))
    outputs = np.zeros((n_samples, 1, nx, ny))
    
    # Generate initial conditions
    for i in range(n_samples):
        init_cond = np.where(X < 0.5, t1, t1 + 2 * (X - 0.5))
        inputs[i, 0] = init_cond
        
        # Simple forward time stepping for output
        outputs[i, 0] = init_cond + alpha * (
            np.roll(init_cond, 1, axis=0) + 
            np.roll(init_cond, -1, axis=0) + 
            np.roll(init_cond, 1, axis=1) + 
            np.roll(init_cond, -1, axis=1) - 
            4 * init_cond
        )
    
    return torch.FloatTensor(inputs).to(device), torch.FloatTensor(outputs).to(device)

In [23]:
# Create model
model = TFNO2d(
    n_modes_width=16,
    n_modes_height=16,
    hidden_channels=32,
    in_channels=1,
    out_channels=1,
    projection_channels=64,
    factorization='tucker',
    implementation='factorized'
).to(device)

# Generate data
train_x, train_y = generate_data(1000)
test_x, test_y = generate_data(100)

# Configure trainer
trainer = Trainer(
    model=model,
    n_epochs=100,
    device=device
)

In [26]:
# Train model
logger.info("Starting training...")

# Create test loader dictionary
test_loaders = {'test': (test_x, test_y)}

# Define optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

# Train model
history = trainer.train(
    (train_x, train_y),
    test_loaders,
    optimizer=optimizer,
    scheduler=scheduler,
    regularizer=None
)

INFO:__main__:Starting training...


AttributeError: 'Tensor' object has no attribute 'items'

In [18]:
# Save and plot results
save_path = "results/neuralop/"
os.makedirs(save_path, exist_ok=True)

plt.figure(figsize=(10, 6))
plt.semilogy(history['train_loss'], label="Training loss")
plt.semilogy(history['test_loss'], label="Testing loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
plt.savefig(os.path.join(save_path, f"neural_operator_baseline_{timestamp}.png"))
plt.show()

# Save model
torch.save(model.state_dict(), os.path.join(save_path, f"model_{timestamp}.pt"))

logger.info(f"Training completed. Final train loss: {history['train_loss'][-1]:.6f}")
logger.info(f"Final test loss: {history['test_loss'][-1]:.6f}")
logger.info(f"Results saved to {save_path}")

NameError: name 'history' is not defined

<Figure size 1000x600 with 0 Axes>